# <center>***Posterior Probability*** <br>
<center>(coding from scratch)


In [1]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-modelling/Churn_Modelling.csv


# Preparing the Dataset for classification

In [2]:
data = pd.read_csv('../input/churn-modelling/Churn_Modelling.csv')

In [3]:
print(data.head)

<bound method NDFrame.head of       RowNumber  CustomerId    Surname  CreditScore Geography  Gender  Age  \
0             1    15634602   Hargrave          619    France  Female   42   
1             2    15647311       Hill          608     Spain  Female   41   
2             3    15619304       Onio          502    France  Female   42   
3             4    15701354       Boni          699    France  Female   39   
4             5    15737888   Mitchell          850     Spain  Female   43   
...         ...         ...        ...          ...       ...     ...  ...   
9995       9996    15606229   Obijiaku          771    France    Male   39   
9996       9997    15569892  Johnstone          516    France    Male   35   
9997       9998    15584532        Liu          709    France  Female   36   
9998       9999    15682355  Sabbatini          772   Germany    Male   42   
9999      10000    15628319     Walker          792    France  Female   28   

      Tenure    Balance  NumOfPro

### Setting the required columns

In [4]:
req_col = ['CreditScore','Geography','Gender','Age','NumOfProducts','Exited']
data = data[req_col]

### Checking the first 5 rows of the DataSet

In [5]:
data.head

<bound method NDFrame.head of       CreditScore Geography  Gender  Age  NumOfProducts  Exited
0             619    France  Female   42              1       1
1             608     Spain  Female   41              1       0
2             502    France  Female   42              3       1
3             699    France  Female   39              2       0
4             850     Spain  Female   43              1       0
...           ...       ...     ...  ...            ...     ...
9995          771    France    Male   39              2       0
9996          516    France    Male   35              1       0
9997          709    France  Female   36              1       1
9998          772   Germany    Male   42              2       1
9999          792    France  Female   28              1       0

[10000 rows x 6 columns]>

### We use this method to gather information about the DataSet

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   CreditScore    10000 non-null  int64 
 1   Geography      10000 non-null  object
 2   Gender         10000 non-null  object
 3   Age            10000 non-null  int64 
 4   NumOfProducts  10000 non-null  int64 
 5   Exited         10000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 468.9+ KB


In [7]:
#assign an arbitrary value according to the size of the dataset to get appropriate sections of the dataset
val = 5
prob = pd.DataFrame(columns = ['Attr','Value','C1','C2','P1','P2'])

### Setting the target
We choose Exited as the target since we are classifying the data into Exited or Did Not Exit the Bank

In [8]:
target = 'Exited'

### Splitting the dataset into Exited and Did Not Exit

In [9]:
#Exited
ex = data[data[target]==1]
#Did Not Exit
nex = data[data[target]==0]

### Checking the shape of both the dataframes

In [10]:
print(nex.shape,ex.shape)

(7963, 6) (2037, 6)


### Choose your input. Uncomment the desired input and Comment the desired one

In [11]:
# For input as Did Not Exit
#input_val = nex.iloc[7005,:]
#print(input_val)


## For input as Exited
input_val = ex.iloc[2005,:]
print(input_val)

CreditScore          467
Geography        Germany
Gender              Male
Age                   28
NumOfProducts          1
Exited                 1
Name: 9825, dtype: object


In [12]:
nex = nex.iloc[0:2040,:]

In [13]:
print(nex.shape,ex.shape)
nex_sh = nex.shape[0]/(ex.shape[0]+nex.shape[0])
ex_sh = ex.shape[0]/(ex.shape[0]+nex.shape[0])
print("P(C1) = {} \n P(C2) = {}".format(ex_sh, nex_sh))

(2040, 6) (2037, 6)
P(C1) = 0.49963208241353935 
 P(C2) = 0.5003679175864606


In [14]:
C1 = []
C2 = []

# Step 2: Calculating P(X/Ci) for classes

In [15]:
for x in req_col:
    
    if (data[x].dtype == np.float64 or data[x].dtype == np.int64):
        if(len(pd.unique(data[x]))>(val)):
            m = data[x].value_counts(bins=3).reset_index().rename(columns={'index':'bin'})
            C1 = []
            C2 = []
            P1 = []
            P2 = []
            
            
            for check in m['bin']:
               
                
                t_c1 = int(ex[(check.left <= ex[x]) & (ex[x] <= check.right)].shape[0])
                t_p1 = int(ex[(check.left <= ex[x]) & (ex[x] <= check.right)].shape[0])/ex.shape[0]
                t_c2 = int(nex[(check.left <= nex[x]) & (nex[x] <= check.right)].shape[0])
                t_p2 = int(nex[(check.left <= nex[x]) & (nex[x] <= check.right)].shape[0])/nex.shape[0]
    
                C1.append(t_c1)
                C2.append(t_c2)
                P1.append(t_p1)
                P2.append(t_p2)
            #print(C1,C2,P1,P2)

            inc = pd.DataFrame({"Attr" : [x,x,x], "Value" : m['bin'], "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
            #print(inc)
            prob = prob.append(inc)
        else:
            C1 = []
            C2 = []
            P1 = []
            P2 = []
            uni = data[x].unique()
            for check in uni:
                if (check in ex[x].value_counts()):
                    C1.append(ex[x].value_counts()[check])
                    #print(C1)
                    P1.append(ex[x].value_counts()[check]/ex.shape[0])
                    
                else:
                    C1.append(0)
                    P1.append(0)
                    
                if (check in nex[x].value_counts()):
                    C2.append(nex[x].value_counts()[check])
                    #print(C2)
                    P2.append(nex[x].value_counts()[check]/nex.shape[0])
                    
                else:
                    C2.append(0)
                    P2.append(0)
                    
                
                    
            inc = pd.DataFrame({"Attr" : [x]*len(uni), "Value" : uni, "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
            prob = prob.append(inc)
                
    else:
        C1 = []
        C2 = []
        P1 = []
        P2 = []
        uni = data[x].unique()
        for check in uni:
            C1.append(ex[x].value_counts()[check])
            #print(C1)
            C2.append(nex[x].value_counts()[check])
            #print(C2)
            P1.append(ex[x].value_counts()[check]/ex.shape[0])
            P2.append(nex[x].value_counts()[check]/nex.shape[0])    
        inc = pd.DataFrame({"Attr" : [x]*len(uni), "Value" : uni, "C1" : C1, "C2" : C2, "P1" : P1, "P2" : P2})
        prob = prob.append(inc)
        

In [16]:
prob = prob.reset_index().drop(['index'],axis = 1)

### Checking and Editing the Input Values to remove the Target Column

In [17]:
print("\nInput Value:")
print(input_val)
print("\nExpected Value:")
input_val = input_val.drop(['Exited'])
print("\nRevised DataFrame")
print(input_val)


Input Value:
CreditScore          467
Geography        Germany
Gender              Male
Age                   28
NumOfProducts          1
Exited                 1
Name: 9825, dtype: object

Expected Value:

Revised DataFrame
CreditScore          467
Geography        Germany
Gender              Male
Age                   28
NumOfProducts          1
Name: 9825, dtype: object


### Calculating the value of P(Ci/X)

In [18]:
fin_P1 = 1
fin_P2 = 1
for inin in input_val.index:
    answer = prob[(prob['Attr'] == inin)]
    #print(answer)
    #print(inin)
    
    for val in answer['Value']:
        #print(type(val))
        if(isinstance(val, int)):
            if(input_val[inin] == val):
                ind = answer.index[answer.Value == val]
                fin_P2*=float(answer['P2'].loc[ind])
                #print(float(answer['P2'].loc[ind]))
                fin_P1*=float(answer['P1'].loc[ind])
                #print(float(answer['P1'].loc[ind]))
            
        else:
            if(input_val[inin] in val):
                ind = answer.index[answer.Value == val]
                fin_P2*=float(answer['P2'].loc[ind])
                #print(float(answer['P2'].loc[ind]))
                fin_P1*=float(answer['P1'].loc[ind])
                #print(float(answer['P1'].loc[ind]))
print("P(X/C1) = {}".format(fin_P1))         
print("P(X/C2) = {}".format(fin_P2))        

P(X/C1) = 0.004935501467808717
P(X/C2) = 0.003915655296871866


# Step 3: Calculating Posterior Probability

In [19]:
print("P(C1/X) = {} * {} = {}".format(fin_P1, ex_sh,fin_P1*ex_sh ))
print("P(C2/X) = {} * {} = {}".format(fin_P2, nex_sh,fin_P2*nex_sh))

P(C1/X) = 0.004935501467808717 * 0.49963208241353935 = 0.0024659348761163495
P(C2/X) = 0.003915655296871866 * 0.5003679175864606 = 0.0019592682868821696


# Conclusion:

In [20]:
if(fin_P1 * ex_sh > fin_P2 * nex_sh):
    print("Conclusion: Since P(X) is constant and common for both the classes and the maximum posterior is P(C1/X) = {}. Hence tuple/unknown sample gets classified as Exited".format(fin_P1*ex_sh))
else:
    print("Conclusion: Since P(X) is constant and common for both the classes and the maximum posterior is P(C2/X) = {}. Hence tuple/unknown sample gets classified as Did Not Exit".format(fin_P2*nex_sh))

Conclusion: Since P(X) is constant and common for both the classes and the maximum posterior is P(C1/X) = 0.0024659348761163495. Hence tuple/unknown sample gets classified as Exited
